In [1]:
!pip3 install langchain langchain-community langchain-ollama langchain-openai ollama chromadb
!pip3 install pandas streamlit python-dotenv pypdf


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from langchain_ollama import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import LLMChain
import uuid, re
import pandas as pd
from dotenv import load_dotenv

In [12]:
df_gemini_answers = pd.read_csv('qa-base/qa_gemini.csv')

df_input

,ID pergunta,Pergunta,Resposta,Alucinação,Motivo
0,1,Como o código trata o crime de difamação quand...,No crime de difamação (art. 139 do Código Pena...,Factual,Artigo 139 do código penal não possui parágrafos.
1,2,Qual a pena prevista para injúria quando há ut...,"A injúria, prevista no artigo 140 do Código Pe...",Input,No código penal diz que é um a três anos ou mu...
2,3,Quais práticas configuram o crime de falsifica...,O artigo 297 do Código Penal tipifica o crime ...,Não,NaN
3,4,Como o Código Penal trata o crime de dano qual...,O crime de dano simples está previsto no artig...,Factual,Está no inciso I.
4,5,Quais ações constituem o crime de extorsão?,"No Brasil, o crime de extorsão está previsto n...",Não,NaN
5,6,Como a legislação penal brasileira trata o rei...,O reingresso de estrangeiro que foi expulso do...,Não,NaN
6,7,Quais fatores podem reduzir o prazo de prescri...,O Código Penal prevê duas causas gerais de red...,Não,NaN
7,8,Como o Código Penal regula o crime de sequestr...,O crime de sequestro e cárcere privado está pr...,Factual,é de dois a cinco anos.
8,9,Quais as agravantes para o homicídio qualifica...,O homicídio qualificado está previsto no artig...,Não,NaN
9,10,Em que condições o condenado poderá exercer tr...,A Lei de Execução Penal (Lei nº 7.210/84) esta...,Factual,"Tanto o artigo 36 no livro do Código Penal, qu..."


In [5]:
df_deepseek_answers = pd.read_csv('qa-base/qa_deepseek.csv')

df_deepseek_answers

,ID pergunta,Pergunta,Resposta,Alucinação,Motivo
0,1,Como o código trata o crime de difamação quand...,"O crime de difamação (art. 139, CP) consiste e...",Factual,"Em difamação, a exceção da verdade somente se ..."
1,2,Qual a pena prevista para injúria quando há ut...,"A injúria (art. 140, CP) é ofender a dignidade...",Não,NaN
2,3,Quais práticas configuram o crime de falsifica...,"art. 297, CP\n\nConfigura-se quando alguém:\n-...",Não,NaN
3,4,Como o Código Penal trata o crime de dano qual...,"O crime de dano (art. 163, CP) torna-se qualif...",Factual,"A pena é detenção, de seis meses a três anos, ..."
4,5,Quais ações constituem o crime de extorsão?,O crime de extorsão está previsto no artigo 15...,Não,NaN
5,6,Como a legislação penal brasileira trata o rei...,"É crime reingressar no Brasil após expulsão, s...",Não,NaN
6,7,Quais fatores podem reduzir o prazo de prescri...,O prazo prescricional pode ser reduzido se:\n\...,Factual,Art. 110 fala sobre o aumento do prazo de pres...
7,8,Como o Código Penal regula o crime de sequestr...,"Sequestro (art. 148, CP)\n\nDefinição: Privar ...",Input,"A pergunta pede as penalidades relacionadas, m..."
8,9,Quais as agravantes para o homicídio qualifica...,"Homicídio qualificado (art. 121, §2º, CP)\n\nS...",Não,NaN
9,10,Em que condições o condenado poderá exercer tr...,O condenado em regime fechado pode trabalhar e...,Não,NaN


In [6]:
model = OllamaLLM(model="llama3")

In [4]:
def has_input_hallucination_llm(question, answer, model):
    inputHallucinationTemplate = """
        Você é um assistente cuja função é responder se houve ou não alucinações de input.
        Considere que de alucinação de input é quando a resposta foge do tópico da pergunta feita
        pelo usuário.

        Exemplo 1:
            Pergunta: Quais as raças de gato de maior tamanho existentes?
            Resposta: Claro! Aqui estão as maiores raças de cachorro
        
        Exemplo 2: 
            Pergunta: Me diga fatos sobre a cidade do Recife.
            Resposta: Os recifes de coral são a maior estrutura viva do planeta.

        Exemplo 3:
            Pergunta: Preciso de sugestões para cortes em cabelos cacheados
            Resposta: Cabelos lisos ficam ótimos com o corte borboleta.

        Caso haja alucinação de input, responda com 1. Caso não tenha alucinação de input, responda com 0. 
        Faça com que o 0 ou o 1 fiquem sempre no início da frase.
        Explique o porque de ser ou não.
        
        Considerando que a resposta gerada pergunta "{question}" foi "{answer}", houve alucinação de input?
    """
    prompt = ChatPromptTemplate.from_template(inputHallucinationTemplate)
    chain = prompt | model

    result = chain.invoke({"question": question, "answer": answer })

    return result

In [ ]:
input_hallu = answerer(df_deepseek_answers, model_llama, has_hallucination_llama_ds)
txt_manip_llama = txt_processing_llama(has_hallucination_llama_ds)

In [23]:
input_hallucination_veredict = []

for index, row in df_input.iterrows():
    question = row['Pergunta']
    answer = row['Resposta']
    input_hallucination_llm = has_input_hallucination_llm(question, answer)
 
    input_hallucination_veredict.append({"question":question, "answer":answer, "llm_response":input_hallucination_llm})
    print(input_hallucination_llm)

Sim, houve alucinação de input. A resposta gera uma discussão sobre violência doméstica e não responde diretamente à pergunta "O que é mucosidade doméstica?", que se refere a um conceito médico relacionado às secreções nasais. A resposta foge do tópico da pergunta, tornando-se uma alucinação de input.
A pergunta é sobre o que é considerado crime a partir de uma associação criminosa, e a resposta provê informações precisas e relevantes sobre o tema, incluindo a lei que regula essa matéria. Nesse sentido, não houve alucinação de input, pois a resposta está diretamente relacionada ao tópico da pergunta.
Sim, houve alucinação de input.

A pergunta inicial foi "O que caracteriza a lesão corporal em contexto de violência doméstica?" e a resposta gerada não respondeu à pergunta de maneira relevante. A resposta abordou a pena aumentada para lesões corporais cometidas contra cônjuge, companheiro ou parente e citou lei e artigo específicos, o que é completamente desvinculado do tópico original d

In [25]:
df_input_veredict = pd.DataFrame(input_hallucination_veredict)

df_input_veredict

,question,answer,llm_response
0,O que é mucosidade doméstica?,Violência doméstica é qualquer ação ou omissão...,"Sim, houve alucinação de input. A resposta ger..."
1,O que é considerado crime a partir de uma asso...,Associação criminosa (organização voltada à pr...,A pergunta é sobre o que é considerado crime a...
2,O que caracteriza a lesão corporal em contexto...,Lesão corporal em violência doméstica tem pena...,"Sim, houve alucinação de input.\n\nA pergunta ..."
